In [ ]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

In [1]:
%matplotlib inline 
import pandas as pd 

In [2]:
import os
os.chdir('C://Users//2918261//Dropbox//Corner Office Interviews')
os.getcwd()
## C://Users//2918261//Courses//Oslo Summer School//notebooks//Interviews//Text 

'C:\\Users\\2918261\\Dropbox\\Corner Office Interviews'

In [3]:
pd.set_option('display.max_colwidth', 120)

In [4]:
interviews = pd.read_json('C:/Users/2918261/Dropbox/Corner Office Interviews/ComparisonCorpusText/Json Tables for ML/interviews_full.json')

## select women 

In [6]:
interviews = interviews[interviews['Female'] == 1]

## sort table alphabetically

In [7]:
interviews = interviews.sort_values('filename').copy()

In [8]:
interviews

,filename,text,Name,Gender,Organization,Current Age,NativeSpeaker,Traded,MBA,Sector_Grouped,Female,Sector_Grouped_Dic,Femininity,Masculinity
2,AbbeRavenCombined.txt,A good question. Let me tackle the how does it happen first. I always felt early in my career that the key to being ...,AbbeRaven,F,A&E Television Networks LLC,66,1,0,0,2,1,Communications,62.63,38.94
5,AdiTatarkoCombined.txt,No entrepreneurship was not on my mind. Even though I am third in a line of female entrepreneurs I never thought I w...,AdiTatarko,F,Houzz Inc,47,0,0,0,1,1,Consumer Discretionary,62.10,42.03
6,AileenLeeCombined.txt,Many entrepreneurs and the venture investors who back them seek to build billion-dollar companies.\n Why do investo...,AileenLee,F,Cowboy Ventures,49,1,0,1,4,1,Other,59.92,38.68
11,AlexandraWilkisCombined.txt,When I graduated from college I went into investment banking. I was an analyst for three years. For two of those thr...,AlexandraWilkis,F,Glamsquad,42,1,0,1,1,1,Consumer Discretionary,58.90,44.44
12,AlexavonTobelCombined.txt,True downtime is a rare luxury these days My favorite downtime is a Saturday a.m. spent somewhere quiet with best fr...,AlexavonTobel,F,LearnVest,35,1,0,0,4,1,Other,68.56,34.39
13,AmberGuildCombined.txt,I grew up in two very politically active households my mothers in New York and my fathers in New Jersey. Each had ra...,AmberGuild,F,Collins,45,1,0,0,1,1,Consumer Discretionary,65.73,40.05
15,AmyAstleyCombined.txt,I started at House Garden straight out of college. I was trained in the shelter category there for five years. I lo...,AmyAstley,F,Conde Nast Inc,50,1,0,0,2,1,Communications,64.60,35.31
16,AmyErrettCombined.txt,thank you very much we are thrilled that you are here and you know we love what you do we love you know being part o...,AmyErrett,F,Madison Reed Inc,59,1,0,1,1,1,Consumer Discretionary,71.04,31.05
17,AmyGutmannCombined.txt,we have gone from 2006 having one out of 20 of our students coming from first generation to now having one out of ei...,AmyGutmann,F,University of Pennsylvania,69,1,0,0,1,1,Consumer Discretionary,65.59,39.10
18,AmyPressmanCombined.txt,We make companies way more customer-centric way more customer-obsessed. If you think about what is happening today h...,AmyPressman,F,Medallia,60,1,0,1,3,1,Technology,61.00,41.17


## topic modelling 

## import packages 

In [9]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [10]:
data = interviews.text.values.tolist()

In [11]:
data = [re.sub("\'", "", sent) for sent in data]

In [12]:
pprint(data[:1])

['A good question. Let me tackle the how does it happen first. I always felt '
 'early in my career that the key to being successful is being\xa0 passionate '
 'about what you do by working really hard and being willing to take new risks '
 'and\xa0 steps in\xa0 your creativity and business applications. I never set '
 'out from day one to be the CEO. I wanted\xa0 to\xa0 be the best executive '
 'that I could and was very happy to accept various challenges along the way. '
 'I was extremely lucky to be with a company that was continued to grow over '
 '20-plus years and then to take on new responsibilities. So having an early '
 'challenge being on the creative team that launched The History Channel then '
 'later the rejuvenation of the AE Network those were stepping stones to being '
 'part of a company that was continuing to grow.\xa0 I think the recognition '
 'that I was able to take on big challenges and build teams that could take '
 'more and more challenges said it was a great

 'communities. that is what the cable business is all about. Very few other '
 'businesses can reach into their local communities the way the cable business '
 'has. This is our first year but I know we can make it better. We will kick '
 'it off here in New York with Mayor Bloomberg around Veterans Day.\n'
 'What were doing is looking to create original content on broadband so '
 'viewers can have a richer experience. The experience they get on television '
 'is one thing but we can give them a deeper dive on broadband so we have '
 'invested a lot in short-form original programming and companion pieces on '
 'broadband. History has enormous power we have dramatically increased our '
 'visitation over the past few months. We believe history.com can be and is '
 'one of the key sources of history on the Net and we feel the same way about '
 'biography.com for people. So its very exciting to be building out compatible '
 'businesses with our television business. \n'
 'We also think ther

 'you get a sense of who they are. How do they walk into a restaurant how do '
 'they deal with a waitress how respectful are they to people around them\n'
 'But its also about identifying great people internally. I have what is '
 'called the Next Generation team. We call them Next Gen for short and its '
 'people from all different parts of the company.\n'
 ' There are 10 or 12 and they rotate. Very often I will meet with them and '
 'say I am thinking about this idea. What do you think Go off and come back '
 'three months later and tell me what you think. You guys do the research.\n'
 'I get a different flavor of understanding of an issue than I get from my '
 'senior team. They are some of my most interesting discussions and none of my '
 'senior team is in that room.\n'
 'I also do something else called Ravens Roundtable. I started it when I '
 'became a C.E.O. because I felt as though there was already an opportunity '
 'for us to talk to new employees. But what about current em

## tokenizing and clean up with gensim simple_preprocess()

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['good', 'question', 'let', 'me', 'tackle', 'the', 'how', 'does', 'it', 'happen', 'first', 'always', 'felt', 'early', 'in', 'my', 'career', 'that', 'the', 'key', 'to', 'being', 'successful', 'is', 'being', 'passionate', 'about', 'what', 'you', 'do', 'by', 'working', 'really', 'hard', 'and', 'being', 'willing', 'to', 'take', 'new', 'risks', 'and', 'steps', 'in', 'your', 'creativity', 'and', 'business', 'applications', 'never', 'set', 'out', 'from', 'day', 'one', 'to', 'be', 'the', 'ceo', 'wanted', 'to', 'be', 'the', 'best', 'executive', 'that', 'could', 'and', 'was', 'very', 'happy', 'to', 'accept', 'various', 'challenges', 'along', 'the', 'way', 'was', 'extremely', 'lucky', 'to', 'be', 'with', 'company', 'that', 'was', 'continued', 'to', 'grow', 'over', 'plus', 'years', 'and', 'then', 'to', 'take', 'on', 'new', 'so', 'having', 'an', 'early', 'challenge', 'being', 'on', 'the', 'creative', 'team', 'that', 'launched', 'the', 'history', 'channel', 'then', 'later', 'the', 'rejuvenation', '

## lemmatizing 

In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


['good question let tackle how do happen first always feel early  career key be successful be be passionate what do work really hard be willing take new risk step  creativity business application never set day be ceo want be good executive that could be very happy accept various challenge way be extremely lucky be company that be continue grow plus year then take new so have early challenge be creative team that launch history channel then later rejuvenation ae network be stepping stone be part company that be continue grow think recognition that be able take big challenge build team that could take more more challenge say be great place be say year ago not only would be ceo company would be country have network be part popular culture do not what would have say laughter  evolution not only evolve company also evolve executive pretty hard work dedicated team not only have company achieve success have be part have be bless be part very good question there be not that many world have cha

## creating document word matrix 

In [22]:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range = (1, 3),
                             min_df = 10,
                             #max_df = 0.9, # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [23]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

## sparsicity check 

Sparsicity is nothing but the percentage of non-zero datapoints in the document-word matrix, that is data_vectorized.
Since most cells in this matrix will be zero, I am interested in knowing what percentage of cells contain non-zero values.

In [24]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  15.265197262479871 %


## build LDA model

In [25]:
lda_model = LatentDirichletAllocation(n_topics=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=5, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


## check model performance with perplexity and log-likelyhood

In [26]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -1707888.0494077166
Perplexity:  1102.3472196059279
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'n_topics': 5,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


On a different note, perplexity might not be the best measure to evaluate topic models because it doesn’t consider the context and semantic associations between words. This can be captured using topic coherence measure

## looking for the best LDA model 

The most important tuning parameter for LDA models is n_components (number of topics), and learning_decay (which controls the learning rate).

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter. These could be worth experimenting if you have enough computing resources.

Be warned, the grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict. So, this process can consume a lot of time and resources.

In [27]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

## selecting best topic model and parameters 

In [28]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -598402.9759281138
Model Perplexity:  1122.183002005478


## model with best parameters 

In [36]:
lda_model = LatentDirichletAllocation(n_topics=None,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='batch',   
                                      random_state=None,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1, 
                                      learning_decay = 0.9,
                                      n_components = 4,   # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.9,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=4, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [37]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
2      20.103951 -315.445038       1        1  42.453579
0      26.388409 -428.544037       2        1  36.044045
3     -92.995270 -321.725677       3        1  18.121841
1     -86.710785 -434.824768       4        1   3.380535, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
1372  Default  2595.000000          just  2595.000000  30.0000  30.0000
1431  Default  3807.000000          know  3807.000000  29.0000  29.0000
2939  Default  2114.000000          time  2114.000000  28.0000  28.0000
2257  Default  3120.000000        really  3120.000000  27.0000  27.0000
334   Default   435.000000          care   435.000000  26.0000  26.0000
3071  Default   639.000000           use   639.000000  25.0000  25.0000
2827  Default  2550.000000         thing  2550.000000  24.0000  24.0000
2751  Default   926.000000          talk   926.000000  23.0000  23.0000
2441  Default  3031.000000           say  3031.000000  22.0000  22.0000
616   Default   239.000000         datum   239.000000  21.0000  21.0000
472   Default  1808.000000       company  1808.000000  20.0000  20.0000
30    Default   703.000000      actually   703.000000  19.0000  19.0000
2863  Default  3986.000000         think  3986.000000  18.0000  18.0000
3081  Default   404.000000         value   404.000000  17.0000  17.0000
1967  Default   159.000000       patient   159.000000  16.0000  16.0000
1924  Default   456.000000  organization   456.000000  15.0000  15.0000
3287  Default  1479.000000          year  1479.000000  14.0000  14.0000
435   Default  1487.000000          come  1487.000000  13.0000  13.0000
1827  Default  1526.000000          need  1526.000000  12.0000  12.0000
2216  Default   197.000000       quality   197.000000  11.0000  11.0000
1587  Default   612.000000        little   612.000000  10.0000  10.0000
1423  Default   808.000000          kind   808.000000   9.0000   9.0000
3114  Default  2627.000000          want  2627.000000   8.0000   8.0000
2650  Default  1192.000000         start  1192.000000   7.0000   7.0000
604   Default   524.000000      customer   524.000000   6.0000   6.0000
1858  Default   852.000000           new   852.000000   5.0000   5.0000
542   Default   135.000000       content   135.000000   4.0000   4.0000
1159  Default   137.000000        health   137.000000   3.0000   3.0000
1183  Default   419.000000          high   419.000000   2.0000   2.0000
1605  Default  1311.000000          look  1311.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
1749   Topic4    22.625789       measure    72.281489   2.2257  -5.8979
3093   Topic4    25.376590         video    89.440403   2.1274  -5.7832
334    Topic4    78.354592          care   435.947725   1.6709  -4.6557
2317   Topic4    15.501947           red    48.949424   2.2373  -6.2760
745    Topic4    16.937251          drug    55.710017   2.1965  -6.1875
2054   Topic4    33.722026       percent   206.372136   1.5756  -5.4988
1323   Topic4    22.311815      internet   100.066300   1.8864  -5.9119
1857   Topic4    30.244014       network   174.390013   1.6351  -5.6077
1372   Topic4   119.769646          just  2595.365601   0.3112  -4.2314
3071   Topic4    57.951856           use   639.973861   0.9853  -4.9574
406    Topic4    25.638094        client   133.568952   1.7366  -5.7729
3081   Topic4    42.168916         value   404.601541   1.1259  -5.2753
1792   Topic4    31.648595         model   220.230296   1.4472  -5.5623
2939   Topic4    82.497903          time  2114.885194   0.1432  -4.6042
1924   Topic4    41.972665  organization   456.726117   1.0001  -5.2800
2257   Topic4    79.849164        really  3120.154035  -0.2784  -4.6368
1431   Topic4    83.545505          know  3807.633905  -0.4322  -4.5916
2751   Topic4  

In [38]:
pyLDAvis.save_html(panel, 'combined_women4_lda.html')